## Solução 02


In [ ]:

import argparse
import sys

# Definir as classes para a AST
class CharNode:
    def __init__(self, value):
        self.value = value

    def match(self, text, pos):
        if pos < len(text) and text[pos] == self.value:
            return pos + 1
        else:
            return None

class DotNode:
    def match(self, text, pos):
        if pos < len(text) and text[pos] != '\n':
            return pos + 1
        else:
            return None

class StarNode:
    def __init__(self, expr):
        self.expr = expr

    def match(self, text, pos):
        while True:
            nextpos = self.expr.match(text, pos)
            if nextpos is None or nextpos == pos:
                return pos
            pos = nextpos

class PlusNode:
    def __init__(self, expr):
        self.expr = expr

    def match(self, text, pos):
        nextpos = self.expr.match(text, pos)
        if nextpos is not None and nextpos != pos:
            return StarNode(self.expr).match(text, nextpos)
        else:
            return None

class OptionalNode:
    def __init__(self, expr):
        self.expr = expr

    def match(self, text, pos):
        nextpos = self.expr.match(text, pos)
        if nextpos is not None:
            return nextpos
        else:
            return pos

class OrNode:
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def match(self, text, pos):
        nextpos = self.left.match(text, pos)
        if nextpos is not None:
            return nextpos
        else:
            return self.right.match(text, pos)

class GroupNode:
    def __init__(self, expr):
        self.expr = expr

    def match(self, text, pos):
        nextpos = self.expr.match(text, pos)
        if nextpos is not None:
            return nextpos
        else:
            return None

class BracketNode:
    def __init__(self, chars, negate=False):
        self.chars = chars
        self.negate = negate

    def match(self, text, pos):
        if pos < len(text) and ((text[pos] in self.chars) != self.negate):
            return pos + 1
        else:
            return None

class RangeNode:
    def __init__(self, start, end):
        self.start = start
        self.end = end

    def match(self, text, pos):
        if pos < len(text) and self.start <= text[pos] <= self.end:
            return pos + 1
        else:
            return None

# Funções auxiliares para parsear a expressão regular
def parse_char(expr, i):
    if i < len(expr) and expr[i] not in '.*+?^$\\[]':
        return CharNode(expr[i]), i + 1


Solução 03

In [ ]:
def conjunto_dominante_simplificado(grafo):
    # Redução do problema de cobertura de vértices para o problema de conjunto dominante
    novo_grafo = {}
    for u, v in grafo.items():
        novo_grafo[u] = set()
        for w in v:
            novo_grafo[u].add((w, u+w))
            novo_grafo[w] = {(u, u+w)}
    
    # Redução do problema de conjunto dominante para SAT
    n = len(novo_grafo)
    m = sum(len(v) for v in novo_grafo.values())
    sat_clauses = []
    
    # Cada vértice deve estar no conjunto dominante ou ter pelo menos um vizinho no conjunto dominante
    for u in range(n):
        sat_clause = []
        for v, literal in novo_grafo[u]:
            sat_clause.append(literal)
        sat_clause.append(-(u+1))
        sat_clauses.append(sat_clause)

Solução 4

In [ ]:
# parte 2

def EMP(L, m, k):
    n = len(L)
    # Cria um conjunto de competências
    competencias = set(range(m))
    # Inicializa uma lista vazia para a equipe
    equipe = []
    # Enquanto ainda não encontrou k empregados
    while len(equipe) < k:
        # Calcula a interseção das competências restantes de cada candidato
        intersecoes = [competencias.intersection(set(L[i])) for i in range(n)]
        # Encontra o índice do candidato com a maior interseção
        i = max(range(n), key=lambda i: len(intersecoes[i]))
        # Se a interseção for vazia, não há solução possível
        if len(intersecoes[i]) == 0:
            return None
        # Adiciona o candidato escolhido para a equipe e remove suas competências
        equipe.append(i)
        competencias -= intersecoes[i]
    return equipe


In [ ]:
#solução 5

parte 01

capacidade, n_estacoes = map(int, input().split())

passageiros = 0

for i in range(n_estacoes):
    sairam, entraram, esperaram = map(int, input().split())
    if passageiros + entraram - sairam > capacidade or passageiros + entraram - sairam < 0:
        print("impossível")
        break
    if esperaram > 0 and passageiros < capacidade:
        print("impossível")
        break
    passageiros += entraram - sairam

if passageiros != 0:
    print("impossível")
else:
    print("possível")


# parte 2

from bisect import bisect_left

def lis(seq):
    # Função para encontrar a subsequência crescente mais longa
    n = len(seq)
    # seq_idx[i] é o índice do menor final para uma subsequência de comprimento i+1
    seq_idx, prev_idx = [0] * n, [None] * n
    # Inicializa a primeira sequência
    seq_idx[0] = 0
    for i in range(1, n):
        if seq[i] > seq[seq_idx[0]]:
            seq_idx[0] = i
        elif seq[i] < seq[seq_idx[-1]]:
            seq_idx[-1] = i
        else:
            pos = bisect_left(seq, seq[i], 0, len(seq_idx))
            seq_idx[pos] = i
        prev_idx[i] = seq_idx[pos-1] if pos > 0 else None

    # Reconstrói a sequência
    longest = [seq[seq_idx[-1]]]
    idx = prev_idx[seq_idx[-1]]
    while idx is not None:
        longest.append(seq[idx])
        idx = prev_idx[idx]
    return len(longest), reversed(longest)

# Lê a entrada
while True:
    try:
        n = int(input())
    except EOFError:
        break
    seq = list(map(int, input().split()))

    # Encontra a subsequência crescente mais longa
    length, indices = lis(seq)

    # Imprime a solução
    print(length)
    print(*[i for i, x in enumerate(seq) if x in indices])
